In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import SVC
import joblib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time

# Ensure TensorFlow uses GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    try:
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(physical_devices)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        print(e)



In [ ]:
# Define paths
data_path = "C:/Users/emir_/Desktop/Yapay Zeka Projesi/data"
directories = ['images_rgb_train', 'images_rgb_val', 'images_thermal_train', 'images_thermal_val']

# Get all categories
all_categories = sorted([d for d in os.listdir(os.path.join(data_path, 'images_rgb_train')) if os.path.isdir(os.path.join(data_path, 'images_rgb_train', d))])

# Get categories from all directories, excluding empty categories for SVM
categories_svm = set()
for directory in directories:
    dir_path = os.path.join(data_path, directory)
    for category in os.listdir(dir_path):
        if os.path.isdir(os.path.join(dir_path, category)):
            category_path = os.path.join(dir_path, category)
            if len(os.listdir(category_path)) > 0:
                categories_svm.add(category)
categories_svm = sorted(categories_svm)

print(f"Number of categories (SVM): {len(categories_svm)}")
print(f"Categories (SVM): {categories_svm}")



In [ ]:
# Function to load images and labels
def load_data(image_path, categories):
    data = []
    labels = []
    for category in categories:
        class_num = categories.index(category)
        path = os.path.join(image_path, category)
        if os.path.exists(path):
            for img in tqdm(os.listdir(path)):
                try:
                    img_array = cv2.imread(os.path.join(path, img))
                    img_array = cv2.resize(img_array, (128, 128))
                    data.append(img_array)
                    labels.append(class_num)
                except Exception as e:
                    pass
    return np.array(data), np.array(labels)

# Load RGB and Thermal data
x_train_rgb, y_train_rgb = load_data(os.path.join(data_path, 'images_rgb_train'), all_categories)
x_val_rgb, y_val_rgb = load_data(os.path.join(data_path, 'images_rgb_val'), all_categories)
x_train_thermal, y_train_thermal = load_data(os.path.join(data_path, 'images_thermal_train'), all_categories)
x_val_thermal, y_val_thermal = load_data(os.path.join(data_path, 'images_thermal_val'), all_categories)
x_test_rgb, y_test_rgb = load_data(os.path.join(data_path, 'video_rgb_test'), all_categories)
x_test_thermal, y_test_thermal = load_data(os.path.join(data_path, 'video_thermal_test'), all_categories)


In [ ]:
# Normalize data
x_train_rgb = x_train_rgb / 255.0
x_val_rgb = x_val_rgb / 255.0
x_train_thermal = x_train_thermal / 255.0
x_val_thermal = x_val_thermal / 255.0
x_test_rgb = x_test_rgb / 255.0
x_test_thermal = x_test_thermal / 255.0

# One-hot encode labels
num_classes = len(all_categories)
y_train_rgb = to_categorical(y_train_rgb, num_classes=num_classes)
y_val_rgb = to_categorical(y_val_rgb, num_classes=num_classes)
y_train_thermal = to_categorical(y_train_thermal, num_classes=num_classes)
y_val_thermal = to_categorical(y_val_thermal, num_classes=num_classes)
y_test_rgb = to_categorical(y_test_rgb, num_classes=num_classes)
y_test_thermal = to_categorical(y_test_thermal, num_classes=num_classes)


In [ ]:
# Cell 4: Define CNN model
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
# Cell 5: Create CNN models
input_shape = (128, 128, 3)

cnn_model_rgb = create_cnn_model(input_shape, num_classes)
cnn_model_thermal = create_cnn_model(input_shape, num_classes)


In [ ]:
# Cell 6: Train or load CNN models
if os.path.exists('cnn_model_rgb.weights.h5') and os.path.exists('cnn_model_thermal.weights.h5'):
    cnn_model_rgb.load_weights('cnn_model_rgb.weights.h5')
    cnn_model_thermal.load_weights('cnn_model_thermal.weights.h5')
else:
    # Train CNN model for RGB images
    history_rgb = cnn_model_rgb.fit(
        x_train_rgb, y_train_rgb,
        epochs=10,
        validation_data=(x_val_rgb, y_val_rgb),
        batch_size=32,
        verbose=1
    )
    
    # Train CNN model for Thermal images
    history_thermal = cnn_model_thermal.fit(
        x_train_thermal, y_train_thermal,
        epochs=10,
        validation_data=(x_val_thermal, y_val_thermal),
        batch_size=32,
        verbose=1
    )

    cnn_model_rgb.save_weights('cnn_model_rgb.weights.h5')
    cnn_model_thermal.save_weights('cnn_model_thermal.weights.h5')


In [ ]:
# Cell 9: Load CNN weights and evaluate
cnn_model_rgb.load_weights('cnn_model_rgb.weights.h5')
cnn_model_thermal.load_weights('cnn_model_thermal.weights.h5')

rgb_cnn_eval = cnn_model_rgb.evaluate(x_val_rgb, y_val_rgb, verbose=1)
print(f"RGB CNN Validation Accuracy: {rgb_cnn_eval[1]:.4f}")

thermal_cnn_eval = cnn_model_thermal.evaluate(x_val_thermal, y_val_thermal, verbose=1)
print(f"Thermal CNN Validation Accuracy: {thermal_cnn_eval[1]:.4f}")

rgb_cnn_test_eval = cnn_model_rgb.evaluate(x_test_rgb, y_test_rgb, verbose=1)
print(f"RGB CNN Test Accuracy: {rgb_cnn_test_eval[1]:.4f}")

thermal_cnn_test_eval = cnn_model_thermal.evaluate(x_test_thermal, y_test_thermal, verbose=1)
print(f"Thermal CNN Test Accuracy: {thermal_cnn_test_eval[1]:.4f}")


In [ ]:
# Flatten the data for SVM
x_train_rgb_flat = x_train_rgb.reshape(x_train_rgb.shape[0], -1)
x_val_rgb_flat = x_val_rgb.reshape(x_val_rgb.shape[0], -1)
x_test_rgb_flat = x_test_rgb.reshape(x_test_rgb.shape[0], -1)


In [ ]:
x_train_thermal_flat = x_train_thermal.reshape(x_train_thermal.shape[0], -1)
x_val_thermal_flat = x_val_thermal.reshape(x_val_thermal.shape[0], -1)
x_test_thermal_flat = x_test_thermal.reshape(x_test_thermal.shape[0], -1)


In [ ]:
# Standardize the data
scaler = StandardScaler()
x_train_thermal_flat = scaler.fit_transform(x_train_thermal_flat)
x_val_thermal_flat = scaler.transform(x_val_thermal_flat)
x_test_thermal_flat = scaler.transform(x_test_thermal_flat)


In [ ]:
scaler = StandardScaler()
x_train_rgb_flat = scaler.fit_transform(x_train_rgb_flat)
x_val_rgb_flat = scaler.transform(x_val_rgb_flat)
x_test_rgb_flat = scaler.transform(x_test_rgb_flat)


In [ ]:
# Apply PCA for dimensionality reduction
pca_components = 20

pca_rgb = PCA(n_components=pca_components).fit(x_train_rgb_flat)
x_train_rgb_pca = pca_rgb.transform(x_train_rgb_flat)
x_val_rgb_pca = pca_rgb.transform(x_val_rgb_flat)
x_test_rgb_pca = pca_rgb.transform(x_test_rgb_flat)


In [ ]:
pca_components = 20

pca_thermal = PCA(n_components=pca_components).fit(x_train_thermal_flat)
x_train_thermal_pca = pca_thermal.transform(x_train_thermal_flat)
x_val_thermal_pca = pca_thermal.transform(x_val_thermal_flat)
x_test_thermal_pca = pca_thermal.transform(x_test_thermal_flat)

In [ ]:
# Custom function to train SVM with progress bar
def train_svm_with_progress_bar(X, y, model_name, kernel='rbf'):
    svm_model = SVC(kernel=kernel, probability=True)
    start_time = time.time()
    svm_model.fit(X, y)
    end_time = time.time()
    print(f"{model_name} Training Time: {end_time - start_time:.2f} seconds")
    joblib.dump(svm_model, f'{model_name}.joblib')
    return svm_model

# Train SVM model for RGB images with progress bar
svm_model_rgb = train_svm_with_progress_bar(x_train_rgb_pca, y_train_rgb.argmax(axis=1), 'svm_model_rgb', kernel='rbf')

# Train SVM model for Thermal images with progress bar
svm_model_thermal = train_svm_with_progress_bar(x_train_thermal_pca, y_train_thermal.argmax(axis=1), 'svm_model_thermal', kernel='rbf')

# Load SVM models
svm_model_rgb = joblib.load('svm_model_rgb.joblib')
svm_model_thermal = joblib.load('svm_model_thermal.joblib')

# Predict and evaluate RGB SVM on validation and test sets
y_pred_rgb_val = svm_model_rgb.predict(x_val_rgb_pca)
y_pred_rgb_test = svm_model_rgb.predict(x_test_rgb_pca)

# Predict and evaluate Thermal SVM on validation and test sets
y_pred_thermal_val = svm_model_thermal.predict(x_val_thermal_pca)
y_pred_thermal_test = svm_model_thermal.predict(x_test_thermal_pca)

accuracy_thermal_val = accuracy_score(y_val_thermal.argmax(axis=1), y_pred_thermal_val)
precision_thermal_val = precision_score(y_val_thermal.argmax(axis=1), y_pred_thermal_val, average='macro', zero_division=0)
recall_thermal_val = recall_score(y_val_thermal.argmax(axis=1), y_pred_thermal_val, average='macro', zero_division=0)
f1_thermal_val = f1_score(y_val_thermal.argmax(axis=1), y_pred_thermal_val, average='macro', zero_division=0)

accuracy_thermal_test = accuracy_score(y_test_thermal.argmax(axis=1), y_pred_thermal_test)
precision_thermal_test = precision_score(y_test_thermal.argmax(axis=1), y_pred_thermal_test, average='macro', zero_division=0)
recall_thermal_test = recall_score(y_test_thermal.argmax(axis=1), y_pred_thermal_test, average='macro', zero_division=0)
f1_thermal_test = f1_score(y_test_thermal.argmax(axis=1), y_pred_thermal_test, average='macro', zero_division=0)

print(f"Thermal SVM Validation Accuracy: {accuracy_thermal_val:.4f}")
print(f"Thermal SVM Validation Precision: {precision_thermal_val:.4f}")
print(f"Thermal SVM Validation Recall: {recall_thermal_val:.4f}")
print(f"Thermal SVM Validation F1 Score: {f1_thermal_val:.4f}")

print(f"Thermal SVM Test Accuracy: {accuracy_thermal_test:.4f}")
print(f"Thermal SVM Test Precision: {precision_thermal_test:.4f}")
print(f"Thermal SVM Test Recall: {recall_thermal_test:.4f}")
print(f"Thermal SVM Test F1 Score: {f1_thermal_test:.4f}")


In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))
plt.plot(history_rgb.history['accuracy'])
plt.plot(history_rgb.history['val_accuracy'])
plt.title('CNN RGB Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(12, 6))
plt.plot(history_rgb.history['loss'])
plt.plot(history_rgb.history['val_loss'])
plt.title('CNN RGB Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation accuracy values for Thermal
plt.figure(figsize=(12, 6))
plt.plot(history_thermal.history['accuracy'])
plt.plot(history_thermal.history['val_accuracy'])
plt.title('CNN Thermal Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values for Thermal
plt.figure(figsize=(12, 6))
plt.plot(history_thermal.history['loss'])
plt.plot(history_thermal.history['val_loss'])
plt.title('CNN Thermal Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [ ]:
# Cell 27: Print Performance Metrics
print("RGB CNN Performance:")
print(f"Accuracy: {history_rgb.history['val_accuracy'][-1]:.4f}")
print(f"Precision: {precision_rgb_val:.4f}")
print(f"Recall: {recall_rgb_val:.4f}")
print(f"F1 Score: {f1_rgb_val:.4f}")

print("\nThermal CNN Performance:")
print(f"Accuracy: {history_thermal.history['val_accuracy'][-1]:.4f}")
print(f"Precision: {precision_thermal_val:.4f}")
print(f"Recall: {recall_thermal_val:.4f}")
print(f"F1 Score: {f1_thermal_val:.4f}")

print("\nRGB SVM Performance:")
print(f"Accuracy: {accuracy_rgb_val:.4f}")
print(f"Precision: {precision_rgb_val:.4f}")
print(f"Recall: {recall_rgb_val:.4f}")
print(f"F1 Score: {f1_rgb_val:.4f}")

print("\nThermal SVM Performance:")
print(f"Accuracy: {accuracy_thermal_val:.4f}")
print(f"Precision: {precision_thermal_val:.4f}")
print(f"Recall: {recall_thermal_val:.4f}")
print(f"F1 Score: {f1_thermal_val:.4f}")
